In [1]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor
from torch.utils.tensorboard import SummaryWriter

pygame 2.6.0 (SDL 2.28.4, Python 3.8.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Import the custom environment and wrapper
from custom_env import CustomEnv
from wrappers import FullyObsSB3MLPWrapper

C:\Users\13769\anaconda3\envs\mini-grid-generation\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\13769\anaconda3\envs\mini-grid-generation\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\13769\anaconda3\envs\mini-grid-generation\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [3]:
# Define the session directory
session_dir = "small_corridor"
os.makedirs(session_dir, exist_ok=True)

# Set the log directory within the session directory
log_dir = os.path.join(session_dir, "logs")
os.makedirs(log_dir, exist_ok=True)

# Set training parameters
total_timesteps = int(1e5)
check_freq = int(1e3)
model_save_path = os.path.join(session_dir, "latest_model")

In [4]:
# Initialize the environment and wrapper
env = CustomEnv(
    txt_file_path='simple_test_corridor.txt',
    display_size=6,
    display_mode="random",
    random_rotate=True,
    random_flip=True,
    custom_mission="Find the key and open the door.",
)
env = FullyObsSB3MLPWrapper(env)

In [5]:
# Load or create a new model
if os.path.exists(f"{model_save_path}.zip"):
    model = PPO.load(model_save_path, env=env)
    print("Loaded model from saved path.")
else:
    model = PPO("MlpPolicy", env, verbose=1)
    print("Initialized new model.")

Using cuda device
Initialized new model.


In [6]:
# Initialize TensorBoard writer
writer = SummaryWriter(log_dir)
# Create evaluation callback
eval_callback = EvalCallback(
    env,
    best_model_save_path=best_model_save_path,
    log_path=log_dir,
    eval_freq=eval_freq,
    deterministic=True,
    render=False
)

# Create checkpoint callback
checkpoint_callback = CheckpointCallback(
    save_freq=save_freq,
    save_path=session_dir,
    name_prefix="rl_model",
    verbose=1
)


In [7]:
# Train the model and log performance
try:
    for i in range(total_timesteps // check_freq):
        # Train the model for a specified number of timesteps
        model.learn(total_timesteps=check_freq, callback=callback)
        
        # Evaluate the model and log performance metrics
        mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
        writer.add_scalar("Evaluation/Mean Reward", mean_reward, i * check_freq)
        writer.add_scalar("Evaluation/Std Reward", std_reward, i * check_freq)

        # Save the latest model
        model.save(model_save_path)
        print(f"Model saved at timestep {i * check_freq}.")
finally:
    # Ensure that the TensorBoard writer is closed properly
    writer.close()

C:\Users\13769\anaconda3\envs\mini-grid-generation\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.ep_info_buffer to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.ep_info_buffer` for environment variables or `env.get_wrapper_attr('ep_info_buffer')` that will search the reminding wrappers.
  logger.warn(


AttributeError: 'CustomEnv' object has no attribute 'ep_info_buffer'